In [3]:
import torch
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [5]:

df_dev = pd.read_json('./Twibot-20/dev.json')

print('loaded df_dev')

loaded df_dev


In [18]:
import re

example_tweet = "RT @andersoncooper Today on @Anderson @PAAK: the debate over #ADHD -- are kids being overmedicated? #Thursday #Anderson\n"

In [26]:
retweetUnameRegex = re.compile(r'(?<=RT @)(\w{1,15})')
mentionsUnameRegex = re.compile(r'(?<!RT @)(?<=@)(\w{1,15})')
# retweetUnameRegex.search(example_tweet)
print(retweetUnameRegex.findall(example_tweet))
print(mentionsUnameRegex.findall(example_tweet))
# find = re.search(r'(?<=RT @)(\w){1,15}', example_tweet)

['andersoncooper']
['Anderson', 'PAAK']


In [53]:
# extracting tweets dataframe from df_dev
# each tweet has a body (text), id (T-userID-tweetNumberByUser), and neighbours - can extract neighbourhoods from userID as well as RT @uname/ @uname for mentions.
## REGULAR EXPRESSIONS
# Regex for capturing Retweet uname: (?<=RT @)(\w{1,15})
# Regex for Mention unames (All inclusive): (?<=@)(\w{1,15})
# Regex for Mention unames WITHOUT retweets: (?<!RT @)(?<=@)(\w{1,15})

# will extract the 3 different sets of neighbours separately then the different version of the dataset can use these freely.
import re


def extractNeighborUnameDict(tweet):
    # extract retweet uname

    retweet_uname = re.findall(r'(?<=RT @)(\w{1,15})', tweet)

    # extract mention unames without retweets
    mentions_no_rt = re.findall(r'(?<!RT @)(?<=@)(\w{1,15})', tweet)

    return {"retweeted": retweet_uname, "mentions": mentions_no_rt}

df_tweet = df_dev[df_dev.tweet.notnull()].apply(lambda x: pd.Series([{"ID": "T"+ str(x['ID']) + "-" + str(i), "Body": tweet, "neighborUsernames": extractNeighborUnameDict(tweet), "tweeterId": x['ID']} for (i,tweet) in enumerate(x['tweet'])]), axis=1)

print(type(df_tweet))
# print(df_tweet[:5])
# unameDict = extractNeighborUnameDict(example_tweet)
# print(unameDict)


<class 'pandas.core.frame.DataFrame'>


In [49]:
print(df_tweet.columns)
print(df_tweet.shape)

print(df_tweet.head().shape)

RangeIndex(start=0, stop=200, step=1)
(2350, 200)
(5, 200)


In [63]:
print(df_tweet.stack())
print(df_tweet.stack().shape)
print(type(df_tweet.stack()))

0     0      {'ID': 'T1224667050301255680-0', 'Body': '@Spa...
      1      {'ID': 'T1224667050301255680-1', 'Body': '@Bar...
      2      {'ID': 'T1224667050301255680-2', 'Body': 'সেদি...
      3      {'ID': 'T1224667050301255680-3', 'Body': 'নিজে...
      4      {'ID': 'T1224667050301255680-4', 'Body': 'ফোন ...
                                   ...                        
2364  195    {'ID': 'T412642667-195', 'Body': 'RT @JaylaCym...
      196    {'ID': 'T412642667-196', 'Body': 'RT @WWE: The...
      197    {'ID': 'T412642667-197', 'Body': 'Cooking, Cla...
      198    {'ID': 'T412642667-198', 'Body': 'RT @TheYBF: ...
      199    {'ID': 'T412642667-199', 'Body': 'RT @TheYBF: ...
Length: 401525, dtype: object
(401525,)
<class 'pandas.core.series.Series'>


In [67]:
df_tweet_alt = pd.DataFrame(list(df_tweet.stack()))
print(df_tweet_alt.shape)

(401525, 4)


In [68]:
print(df_tweet_alt.columns)

Index(['ID', 'Body', 'neighborUsernames', 'tweeterId'], dtype='object')


In [75]:
## Final code for extracting the tweets dataframe from df_dev

df_tweet = df_dev[df_dev.tweet.notnull()].apply(lambda x: pd.Series([{"ID": "T"+ str(x['ID']) + "-" + str(i), "Body": tweet, **extractNeighborUnameDict(tweet), "tweeterId": x['ID']} for (i,tweet) in enumerate(x['tweet'])]), axis=1)
df_tweet = pd.DataFrame(list(df_tweet.stack()))

In [76]:
print(df_tweet.columns)

Index(['ID', 'Body', 'retweeted', 'mentions', 'tweeterId'], dtype='object')
